In [1]:
# %load_ext autoreload
# %autoreload 2
import podpac
import numpy as np

/Users/marcus/computing/python/anaconda3/envs/podpac2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
username=''
password=''
smap = podpac.datalib.smap.SMAP(product='SPL3SMP', username=username, password=password)

## SMAP source example

In [3]:
smap.source_coordinates

Coordinates
	time: ArrayCoordinates1d(time): Bounds[2015-03-31, 2018-06-03], N[1157], ctype['midpoint']

In [4]:
nc = smap.find_coordinates()[0]
nc

Coordinates
	time: ArrayCoordinates1d(time): Bounds[2015-03-31, 2018-06-03], N[1157], ctype['midpoint']
	lat: ArrayCoordinates1d(lat): Bounds[-83.63197326660156, 83.63196563720703], N[406], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-179.8129119873047, 179.8129119873047], N[964], ctype['midpoint']

In [5]:
c = podpac.coordinates.merge_dims([nc.drop('time'), podpac.Coordinates(['2018-05-19T12'], ['time'])])
c

Coordinates
	lat: ArrayCoordinates1d(lat): Bounds[-83.63197326660156, 83.63196563720703], N[406], ctype['midpoint']
	lon: ArrayCoordinates1d(lon): Bounds[-179.8129119873047, 179.8129119873047], N[964], ctype['midpoint']
	time: ArrayCoordinates1d(time): Bounds[2018-05-19T12, 2018-05-19T12], N[1], ctype['midpoint']

In [6]:
smap.source_coordinates.intersect(c, outer=True)

Coordinates
	time: ArrayCoordinates1d(time): Bounds[2018-05-19, 2018-05-20], N[2], ctype['midpoint']

In [7]:
smap.interpolation

'nearest'

In [9]:
out = smap.eval(c)  # <-- returns all nans
out.data

array([[[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       ...,

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]],

       [[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]]])

In [15]:
np.where(np.isnan(out.data))

(array([  0,   0,   0, ..., 405, 405, 405]),
 array([  0,   1,   2, ..., 961, 962, 963]),
 array([0, 0, 0, ..., 0, 0, 0]))

## Recreate Simple Example

In [53]:
from podpac.coordinates import Coordinates, clinspace, UniformCoordinates1d
import numpy as np

a = Coordinates([clinspace('2015-03-31', '2018-06-03', 1157)], ['time'])
b = Coordinates([clinspace(-83.63197326660156, 83.63196563720703, 406), 
                 clinspace(-179.8129119873047, 179.8129119873047, 964), 
                 '2018-05-19T12'], ['lat', 'lon','time'])

c = Coordinates([clinspace(-83.63197326660156, 83.63196563720703, 406), 
                 clinspace(-179.8129119873047, 179.8129119873047, 964), 
                 clinspace('2015-05-19T12', '2018-05-20T20', 1157)], ['lat', 'lon','time'])

d = Coordinates([clinspace(0, 10, 11)], ['lat'])
e = Coordinates([[5.2]], ['lat'])

a.intersect(b, outer=True)
# d.intersect(e)

Coordinates
	time: UniformCoordinates1d(time): Bounds[2018-05-19, 2018-05-20], N[2], ctype['midpoint']

In [47]:
self = a['time']
bounds = b['time'].bounds
outer = True

lo = max(bounds[0], self.bounds[0])
hi = min(bounds[1], self.bounds[1])

imin = int(np.ceil((lo - self.bounds[0]) / np.abs(self.step)))
imax = int(np.floor((hi - self.bounds[0]) / np.abs(self.step)))

if outer:
    imin -= 1
    imax += 1

imax = np.clip(imax+1, 0, self.size)
imin = np.clip(imin, 0, self.size)

# empty case
if imin >= imax:
    ret = self._select_empty(return_indices)

if self.is_descending:
    imax, imin = self.size - imin, self.size - imax

start = self.start + imin*self.step
stop = self.start + (imax-1)*self.step
c = UniformCoordinates1d(start, stop, self.step, **self.properties)

In [51]:
bounds

array(['2018-05-19T12', '2018-05-19T12'], dtype='datetime64[h]')

In [25]:
a = Coordinates([['2015-05-19', '2018-05-20']], ['time'])
b = Coordinates([clinspace(0, 10, 11), 
                 clinspace(0, 10, 11), 
                 '2018-05-19T12'], ['lat', 'lon','time'])

a.intersect(b, outer=True)
# d.intersect(e)

Coordinates
	time: ArrayCoordinates1d(time): Bounds[2015-05-19, 2018-05-20], N[2], ctype['midpoint']

In [15]:
self = a['time']
bounds = b['time'].bounds

In [21]:
gt = np.where(self.coords >= bounds[0])[0]
lt = np.where(self.coords <= bounds[1])[0]

In [23]:
gt[0]

1

In [24]:
lt[0]

0